In [21]:
import mne
# %matplotlib inline
%matplotlib tk
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
data_path = r'..\..\..\app\eeg\static\eeg_data\raw\aud-norm\11.set'

In [22]:
mne.set_log_level('ERROR')
raw = mne.io.read_raw_eeglab(data_path, preload=True)
raw.crop(tmax=30).load_data()
fig = raw.plot_sensors(show_names=True, show=False)

# fig_raw = raw.plot(duration=30,remove_dc=False, title='Raw EEG Data')
# fig_raw.suptitle("Raw data")

In [ ]:
mne.set_log_level('ERROR')
raw = mne.io.read_raw_eeglab(data_path, preload=True)
raw.crop(tmax=30).load_data()
fig = raw.plot_sensors(show_names=True, show=False)


Text(0.5, 0.98, 'Raw data')

## Частотная фильтрация
применяется фильтр с конечной импульсивной характеристикой FIR.
оставляем частоты в диапазоне от 16 до 22 Гц

In [ ]:
# Убираем низкие частоты + ставим ограничение до 45 Гц
raw_filtered_low_freq_drifts = raw.copy().filter(l_freq=16.0,h_freq=22.0)

In [ ]:
# Построение графика исходных данных
fig_raw = raw.plot(duration=30,remove_dc=False, title='Raw EEG Data')
fig_raw.suptitle("Raw data")

# Построение графика после применения частотной фильтрации
fig = raw_filtered_low_freq_drifts.plot(duration=30,remove_dc=False)
fig.suptitle("Filtered low frequency") 

In [ ]:
fig_raw = raw.plot_psd(fmin=1, fmax=40, show=True)
fig_raw.suptitle("Raw data")

fig = raw_filtered_low_freq_drifts.plot_psd(fmin=16, fmax=22, show=True)
fig.suptitle("Filtered low frequency") 

## Устранение глазодвигательных артефактов
Вручную выставляем какой-то канал как EOG, чтобы потом применить автоматический поиск артефактов с помощью ICA

In [ ]:
from mne.preprocessing import ICA

raw_filtered_low_freq_drifts.set_channel_types({'E127': 'eog'})

ica = ICA(n_components=20, random_state=97)
ica.fit(raw_filtered_low_freq_drifts)

eog_indices, _ = ica.find_bads_eog(raw_filtered_low_freq_drifts, threshold=0.5)
ica.exclude = eog_indices

raw_clean = ica.apply(raw_filtered_low_freq_drifts)
fig = raw_clean.plot()
fig.suptitle("Filtered ICA") 


In [ ]:
fig = raw_clean.plot_psd(fmin=1, fmax=40, show=True)
fig.suptitle("Filtered ICA") 